<a href="https://colab.research.google.com/github/vchraf/GeniusLyricsScraper/blob/main/Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import urllib.request
import urllib.parse
import json
import re
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
BASE = "https://api.genius.com"
CLIENT_ACCESS_TOKEN = "-q1tRGBZMEOk6JewZCC_KWZBxyFSg9nccGlX11Cb3MxpGpzWG4FBSJIXCJS33D3x"

In [ ]:
def getJsonData(query, page = 1 ):
  f_query  = f"{BASE}/{query}"
  token = f"Bearer {CLIENT_ACCESS_TOKEN}"
  headers = {"Authorization": token}
  params = {'page': page}
  response = requests.get(url=f_query, params=params, headers=headers)
  return response.json()

In [ ]:
def getArtistID(artistName):
  try:
    return getJsonData(f"search?q={artistName}")['response']['hits'][0]['result']['primary_artist']['id']
  except:
    return -1

In [ ]:
def getListOfSongsIDs(artistID,Max = 2):
  if artistID != -1:
    query = f"artists/{artistID}/songs/"
    page = 1
    artistJson = []
    while True:
      jsonData = getJsonData(query,page)['response']['songs']
      if jsonData: artistJson += jsonData
      else: break
      if page == Max:break
      else: page += 1

    return [song["id"] for song in artistJson if song["primary_artist"]["id"] == artistID]

In [ ]:
def getLyric(SongID):
  query = f"songs/{SongID}"
  jsonData = getJsonData(query)
  URL = "http://genius.com" + jsonData['response']['song']['path']
  page = requests.get(URL)
  html = BeautifulSoup(page.text, "html.parser")
  js = html.find_all("script")[16]
  txt = str(js)[str(js).find('\\"html\\":\\"<p><b>')+11 : str(js).find('\\"children\\":[{\\"children\\') - 2]
  txt = re.sub('<.*?>', '',txt)
  txtli = txt.split('\\n')
  return URL,[t.replace("\\","") for t in txtli if t != '\\' ][1:]

In [ ]:
def writeOnFile_txt(path, input):
  textfile = open(path, "a")
  for element in input:
    textfile.write(element + "\n")
  textfile.close()

In [ ]:
for art in artList.text.values:
  listOfSongs = getListOfSongsIDs(getArtistID(art))
  for id in listOfSongs:
    writeOnFile_txt("Lyr.txt", getLyric(id)[1])